In [ ]:
# Необходимые библиотеки
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind

# Отображени
pd.set_option('display.max_rows', 50) # выведем больше строк
pd.set_option('display.max_columns', 50) # выведем больше колонок

# Загрузка данных

# 1 school — аббревиатура школы, в которой учится ученик
# 2 sex — пол ученика ('F' - женский, 'M' - мужской)
# 3 age — возраст ученика (от 15 до 22)
# 4 address — тип адреса ученика ('U' - городской, 'R' - за городом)
# 5 famsize — размер семьи('LE3' <= 3, 'GT3' >3)
# 6 Pstatus — статус совместного жилья родителей ('T' - живут вместе 'A' - раздельно)
# 7 Medu — образование матери (0 - нет, 1 - 4 класса, 2 - 5-9 классы, 3 - среднее специальное или 11 классов, 4 - высшее)
# 8 Fedu — образование отца (0 - нет, 1 - 4 класса, 2 - 5-9 классы, 3 - среднее специальное или 11 классов, 4 - высшее)
# 9 Mjob — работа матери ('teacher' - учитель, 'health' - сфера здравоохранения, 'services' - гос служба, 'at_home' - не работает, 'other' - другое)
# 10 Fjob — работа отца ('teacher' - учитель, 'health' - сфера здравоохранения, 'services' - гос служба, 'at_home' - не работает, 'other' - другое)
# 11 reason — причина выбора школы ('home' - близость к дому, 'reputation' - репутация школы, 'course' - образовательная программа, 'other' - другое)
# 12 guardian — опекун ('mother' - мать, 'father' - отец, 'other' - другое)
# 13 traveltime — время в пути до школы (1 - <15 мин., 2 - 15-30 мин., 3 - 30-60 мин., 4 - >60 мин.)
# 14 studytime — время на учёбу помимо школы в неделю (1 - <2 часов, 2 - 2-5 часов, 3 - 5-10 часов, 4 - >10 часов)
# 15 failures — количество внеучебных неудач (n, если 1<=n<3, иначе 4)
# 16 schoolsup — дополнительная образовательная поддержка (yes или no)
# 17 famsup — семейная образовательная поддержка (yes или no)
# 18 paid — дополнительные платные занятия по математике (yes или no)
# 19 activities — дополнительные внеучебные занятия (yes или no)
# 20 nursery — посещал детский сад (yes или no)
# 21 higher — хочет получить высшее образование (yes или no)
# 22 internet — наличие интернета дома (yes или no)
# 23 romantic — в романтических отношениях (yes или no)
# 24 famrel — семейные отношения (от 1 - очень плохо до 5 - очень хорошо)
# 25 freetime — свободное время после школы (от 1 - очень мало до 5 - очень мого)
# 26 goout — проведение времени с друзьями (от 1 - очень мало до 5 - очень много)
# 27 health — текущее состояние здоровья (от 1 - очень плохо до 5 - очень хорошо)
# 28 absences — количество пропущенных занятий
# 29 score — баллы по госэкзамену по математике

df = pd.read_csv('stud_math.csv')
print(df)
print(df.info())
print(df.columns)
# Выводы:
# dtypes: float64(12), int64(1), object(17)
# "studytime, granular" - нет расшифровки столбца в задании

In [ ]:
# Здесь будут функции
# Функция для boxplot
def get_boxplot(column):
    fig, ax = plt.subplots(figsize = (14, 4))
    sns.boxplot(x=column, y='score', 
                data=df.loc[df.loc[:, column].isin(df.loc[:, column].value_counts().index[:10])],
               ax=ax)
    plt.xticks(rotation=45)
    ax.set_title('Boxplot for ' + column)
    plt.show()

# Замена пустых ячеек на 'empty'
def empty_change(column):
    df[column] = df[column].apply(lambda x: 'empty' if pd.isnull(x) else x)

# Тест Стьюдента
def student_test(column):
    cols = df.loc[:, column].value_counts().index[:10]
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(df.loc[df.loc[:, column] == comb[0], 'score'], 
                        df.loc[df.loc[:, column] == comb[1], 'score']).pvalue \
            <= 0.05/len(combinations_all): # Учли поправку Бонферони
            print('Найдены статистически значимые различия для колонки', column)
            break
    print(column, ' - End')

In [ ]:
# Проверим основной столбец score
# score - 389 непустых значений из 395 (строки с пустыми значениями в колонке score нужно убрать, т. к. анализируем датасет относительно данной колонки)
df = df.dropna(subset=['score'], axis=0).reset_index(drop=True)

df.score.hist()
df.score.describe()
# Значения распределены между 20 и 100. Есть выброс - значение 0 (скорее всего эти люди не сдавали экзамен, иначе были бы оценки между 0 и 20)
# Возможно, также, что методика оценки подразумевает ставить 0, если набранно менее 20 баллов, но нам об этом неизвестно.
# Удалим строки с нулевыми оценками из анализа
df = df[df.score != 0].reset_index(drop=True)

In [ ]:
# 1 school — аббревиатура школы, в которой учится ученик
# Пустых значений нет

print(df.school.value_counts())
# В датасете 2 школы
# GP 311
# MS 41

In [ ]:
get_boxplot('school')
# В школе GP выше разброс оценок, но общий уровень подготовки выше, чем в школе MS

In [ ]:
# 2 sex — пол ученика ('F' - женский, 'M' - мужской)
# Пустых значений нет

print(df.sex.value_counts())
# F	182
# M	170

In [ ]:
get_boxplot('sex')
# Мальчики в среднем лучше сдают экзамен по метематике

In [ ]:
# 3 age — возраст ученика (от 15 до 22)
# Пустых значений нет
# В колонке представлены дискретные значения возраста (8 значений от 15 до 22), поэтому пока посмотрим на данные, как на номинативные переменные

df.boxplot(column=['age'])
get_boxplot('age')

# Ср уровень оценок снижается с 15 до 17 лет (в 17 лет в среднем экзамен сдают хуже), с 18 до 20 лет ср уровень оценок повышается. В 20 лет максимальный уровень оценок
# Есть выбросы, удалим их из анализа
df = df.loc[df.age.between(15, 20)].reset_index(drop=True)

In [ ]:
print(df.info())
# Непустые значения 336 из 350. 4% - пустых значений.
empty_change('address')

print(df.address.value_counts())
# U 264
# R 72
# empty 14

get_boxplot('address')
# Пока очевидных выводов по данной колонке нет

In [ ]:
# 5 famsize — размер семьи('LE3' <= 3, 'GT3' >3)
print(df.info())
# Непустые значения 326 из 350. 6.86% - пустых значений.
empty_change('famsize')

print(df.famsize.value_counts())
# GT3 226
# LE3 100
# empty 24

get_boxplot('famsize')
# Пока очевидных выводов по данной колонке нет

In [ ]:
# 6 Pstatus — статус совместного жилья родителей ('T' - живут вместе 'A' - раздельно)
print(df.info())
# Непустые значения 309 из 350. 11.7% - пустых значений.
empty_change('Pstatus')

print(df.Pstatus.value_counts())
# T 275
# empty 41
# A 34

get_boxplot('Pstatus')
# Выводы: Много незаполненных данных (больше, чем одна из двух заполненных групп)
# Диаграмма показывает, что данный параметр практически не влияет на итоговый результат
# Столбец можно удалить
df.drop(['Pstatus'], inplace = True, axis = 1)

In [ ]:
# 7 Medu — образование матери (0 - нет, 1 - 4 класса, 2 - 5-9 классы, 3 - среднее специальное или 11 классов, 4 - высшее)
print(df.info())
# Непустые значения 348 из 350. 0.57% - пустых значений (мало, можно их не учитывать)

print(df.Medu.value_counts())
# 4.0    125
# 2.0     88
# 3.0     86
# 1.0     46
# 0.0      3

get_boxplot('Medu')
# Пока очевидных выводов по данной колонке нет

In [ ]:
# 8 Fedu — образование отца (0 - нет, 1 - 4 класса, 2 - 5-9 классы, 3 - среднее специальное или 11 классов, 4 - высшее))
print(df.info())
# Непустые значения 329 из 350. 6% - пустых значений.

print(df.Fedu.value_counts())
# 2.0     94
# 3.0     89
# 4.0     80
# 1.0     63
# 0.0      2
# 40.0     1 - ошибка в датасете, скорее всего имелось в виду 4

df.at[df[df.Fedu == 40]['Fedu'].index[0], 'Fedu'] = 4
print(df.Fedu.value_counts())
# 2.0    94
# 3.0    89
# 4.0    81
# 1.0    63
# 0.0     2

get_boxplot('Fedu')
# Пока очевидных выводов по данной колонке нет

In [ ]:
# 9 Mjob — работа матери ('teacher' - учитель, 'health' - сфера здравоохранения, 'services' - гос служба, 'at_home' - не работает, 'other' - другое)
print(df.info())
# Непустые значения 333 из 350. 4.86% - пустых значений.
empty_change('Mjob')

print(df.Mjob.value_counts())
# other       114
# services     89
# teacher      51
# at_home      49
# health       30
# empty        17

get_boxplot('Mjob')
# Уровень сдачи экзамена в среднем самый высокий, если сфера работы матери 'health' - сфера здравоохранения

In [ ]:
# 10 Fjob — работа отца ('teacher' - учитель, 'health' - сфера здравоохранения, 'services' - гос служба, 'at_home' - не работает, 'other' - другое)
print(df.info())
# Непустые значения 318 из 350. 9.14% - пустых значений.
empty_change('Fjob')

print(df.Fjob.value_counts())
# other       175
# services     88
# empty        32
# teacher      26
# health       15
# at_home      14

get_boxplot('Fjob')
# Очень много незаполненных значений, но они приблизительно равномерно распределены на всем диапазоне оценок, что не повлияет на анализ, если ими принебречь
# В значении 'other' есть выбросы
# Баллы у учеников, у отца которых образование 'teacher' - учитель, в среднем самые высокие.

In [ ]:
# 11 reason — причина выбора школы ('home' - близость к дому, 'reputation' - репутация школы, 'course' - образовательная программа, 'other' - другое)
print(df.info())
# Непустые значения 318 из 350. 4.29% - пустых значений.
empty_change('reason')

print(df.reason.value_counts())
# course        118
# reputation     96
# home           89
# other          32
# empty          15

get_boxplot('reason')
# Есть выбросы в значение 'other'

In [ ]:
# 12 guardian — опекун ('mother' - мать, 'father' - отец, 'other' - другое)
print(df.info())
# Непустые значения 322 из 350. 8% - пустых значений.
empty_change('guardian')

print(df.guardian.value_counts())
# mother    224
# father     78
# empty      28
# other      20

get_boxplot('guardian')
# Пустных значений хоть и много, в целом не влияют на результат
# Много значений 'other' - которые непонятно, как интерпретировать. Уровень медианного значения, примерно такой же, но нижняя граница оценок в этой группе выше.
# Если опекун отец, в среднем экзамен сдается лучше и разброс оценок меньше.

In [ ]:
# 13 traveltime — время в пути до школы (1 - <15 мин., 2 - 15-30 мин., 3 - 30-60 мин., 4 - >60 мин.)
print(df.info())
# Непустые значения 322 из 350. 6.57% - пустых значений.
empty_change('traveltime')

print(df.traveltime.value_counts())
# 1.0      217
# 2.0       86
# empty     23
# 3.0       17
# 4.0        7

get_boxplot('traveltime')
# Пустых значений довольно много
# Большинство значений находятся в одной группе
# Диаграмма показывает отстутствие значимого влияния данного параметра на итоговый результат
# Данный столбец можно удалить из анализа
df.drop(['traveltime'], inplace = True, axis = 1)

In [ ]:
# 14 studytime — время на учёбу помимо школы в неделю (1 - <2 часов, 2 - 2-5 часов, 3 - 5-10 часов, 4 - >10 часов)
print(df.info())
# Непустые значения 343 из 350. 2% - пустых значений.

print(df.studytime.value_counts())
# 2.0      173
# 1.0       88
# 3.0       58
# 4.0       24

get_boxplot('studytime')
# Пока очевидных выводов по данной колонке нет

In [ ]:
# 15 failures — количество внеучебных неудач (n, если 1<=n<3, иначе 4)
print(df.info())
# Непустые значения 322 из 350. 5.71% - пустых значений.
empty_change('failures')

print(df.failures.value_counts())
# 0.0      273
# 1.0       38
# empty     20
# 2.0       11
# 3.0        8

get_boxplot('failures')
# Значения 4 (как в описании) отсутствует. Неописанные значения 0. Возможно проставили 0 вместо 4.
# Количесво пустых значений превышает кол-во значение в двух признаках. Подавляющее большинство данных имеет значение 0.
# Колонка не представляет интереса для аналаза.
df.drop(['failures'], inplace = True, axis = 1)

In [ ]:
# 16 schoolsup — дополнительная образовательная поддержка (yes или no)
print(df.info())
# Непустые значения 322 из 350. 2.29% - пустых значений.
empty_change('schoolsup')

print(df.schoolsup.value_counts())
# no       294
# yes       48
# empty      8

get_boxplot('schoolsup')
# Когда дополнительной образовательной поддержки нет в среднем экзамен сдается лучше, но и разброс оценок больше

In [ ]:
# 17 famsup — семейная образовательная поддержка (yes или no)
print(df.info())
# Непустые значения 314 из 350. 10.29% - пустых значений.
empty_change('famsup')

print(df.famsup.value_counts())
# yes      196
# no       118
# empty     36

get_boxplot('famsup')
# Довольно много незаполненных значений
# Судя по графикам, семейная образовательная поддержка значимого влияния на результат не оказывает
df.drop(['famsup'], inplace = True, axis = 1)

In [ ]:
# 18 paid — дополнительные платные занятия по математике (yes или no)
print(df.info())
# Непустые значения 314 из 350. 10.29% - пустых значений.
empty_change('paid')

print(df.paid.value_counts())
# no       168
# yes      146
# empty     36

get_boxplot('paid')
# Довольно много незаполненных значений
# Судя по графикам, дополнительные платные занятия значимого влияния на результат не оказывает
df.drop(['paid'], inplace = True, axis = 1)

In [ ]:
# 19 activities — дополнительные внеучебные занятия (yes или no)
print(df.info())
# Непустые значения 338 из 350. 3.43% - пустых значений.
empty_change('activities')

print(df.activities.value_counts())
# yes      172
# no       166
# empty     12

get_boxplot('activities')
# При примерно равном медианном значении, общий уровень оценок при наличии дополнительных внеучебных занятий выше (выше среднее значение, 1 и 3 квартили)

In [ ]:
# 20 nursery — посещал детский сад (yes или no)
print(df.info())
# Непустые значения 338 из 350. 3.43% - пустых значений.
empty_change('nursery')

print(df.nursery.value_counts())
# yes      272
# no        66
# empty     12

get_boxplot('nursery')

In [ ]:
# Неописанный столбец 'studytime, granular'
# Судя по названию, возможно есть корреляция со столбцом studytime

df.corr().loc['studytime', 'studytime, granular']
# -0.9999999999999991
# Практически 100% корреляция, столбец можно удалить.

df.drop(['studytime, granular'], inplace = True, axis = 1)

In [ ]:
# 21 higher — хочет получить высшее образование (yes или no)
print(df.info())
# Непустые значения 343 из 350. 5.14% - пустых значений.
empty_change('higher')

print(df.higher.value_counts())
# yes      320
# empty     18
# no        12

get_boxplot('higher')
# Уровень сдачи экзамена у тех, кто не хочет получать высшее образование ниже.
# Значений 'no' недостаточно для учета в анализе. Их меньше, чем незаполненных значений. И всего 3% от заполненных
df.drop(['higher'], inplace = True, axis = 1)

In [ ]:
# 22 internet — наличие интернета дома (yes или no)
print(df.info())
# Непустые значения 321 из 350. 8.29% - пустых значений.
empty_change('internet')

print(df.internet.value_counts())
# yes      273
# no        48
# empty     29

get_boxplot('internet')
# При примерно равной медиане, уровень сдачи экзамена у тех, у кого есть интернет выше. Разброс тоже выше из-за большего количества значений

In [ ]:
# 23 romantic — в романтических отношениях (yes или no)
print(df.info())
# Непустые значения 321 из 350. 8.29% - пустых значений.
empty_change('romantic')

print(df.romantic.value_counts())
# no       218
# yes      103
# empty     29

get_boxplot('romantic')
# При примерно одинаковом медианном значении, общий уровень оценки выше у тех, у кого нет романтических отношений

In [ ]:
# 24 famrel — семейные отношения (от 1 - очень плохо до 5 - очень хорошо)
print(df.info())
# Непустые значения 326 из 350. 6,68% - пустых значений.

df.famrel.hist()
df.famrel.describe()
# Есть выброс - значение -1. Скорее всего, хотели написать значение 1.
# Основные значения от 3 до 5

df.at[df[df.famrel == -1]['famrel'].index[0], 'famrel'] = 1

# count    326.000000
# mean       3.941718
# std        0.894243
# min        1.000000
# 25%        4.000000
# 50%        4.000000
# 75%        5.000000
# max        5.000000

In [ ]:
# 25 freetime — свободное время после школы (от 1 - очень мало до 5 - очень мого)
print(df.info())
# Непустые значения 342 из 350. 2.29% - пустых значений.

df.freetime.hist()
df.freetime.describe()
# Выбросов нет
# count    342.000000
# mean       3.228070
# std        1.001773
# min        1.000000
# 25%        3.000000
# 50%        3.000000
# 75%        4.000000
# max        5.000000

In [ ]:
# 26 goout — проведение времени с друзьями (от 1 - очень мало до 5 - очень много)
print(df.info())
# Непустые значения 344 из 350. 1.71% - пустых значений.

df.goout.hist()
df.goout.describe()
# Выбросов нет
# count    344.000000
# mean       3.081395
# std        1.090271
# min        1.000000
# 25%        2.000000
# 50%        3.000000
# 75%        4.000000
# max        5.000000

In [ ]:
# 27 health — текущее состояние здоровья (от 1 - очень плохо до 5 - очень хорошо)
print(df.info())
# Непустые значения 337 из 350. 3.71% - пустых значений.

df.health.hist()
df.health.describe()
# Выбросов нет
# count    337.000000
# mean       3.525223
# std        1.410036
# min        1.000000
# 25%        3.000000
# 50%        4.000000
# 75%        5.000000
# max        5.000000

In [ ]:
# 28 absences — количество пропущенных занятий
print(df.info())
# Непустые значения 340 из 350. 2.86% - пустых значений.

df.absences.hist()
df.absences.describe()
# Есть выбросы.
# Очевидно, что нижняя граница даннаго параметра 0.
# Отфильтруем значения выше perc75 + 1.5*IQR
IQR = df.absences.quantile(0.75) - df.absences.quantile(0.25)
perc75 = df.absences.quantile(0.75)
df = df.loc[df.absences <= perc75 + 1.5*IQR].reset_index()

In [ ]:
df.absences.hist()
df.absences.describe()

# count    313.000000
# mean       4.651757
# std        4.390053
# min        0.000000
# 25%        2.000000
# 50%        4.000000
# 75%        7.000000
# max       17.000000

In [ ]:
# Посмотрим корреляцию числовых столбцов

df.corr()
# Средний уровень корреляции с оценкой за экзамен наблюдается у следующих параметров:
# age - возраст (отрицаательная корреляция)
# Medu и Fedu - уровень образования родителей. При этом уровень образония матери больше влияет на оценку, чем уровень образования отца
# studytime - время на учёбу помимо школы
# goout - проведение времени с друзьями (отрицаательная корреляция)

In [ ]:
# Коррелиция между Medu и Fedu около 61%. Можно заполнить пропуски в Fedu из Medu и наоборот.
for i in range(0, len(df)):
    if pd.isnull(df.loc[i]['Medu']) & pd.isnull(df.loc[i]['Fedu']):
        continue
    if pd.isnull(df.loc[i]['Medu']) | pd.isnull(df.loc[i]['Fedu']):
        if pd.isnull(df.loc[i]['Medu']):
            df.at[i, 'Medu'] = df.loc[i]['Fedu']
        else:
            df.at[i, 'Fedu'] = df.loc[i]['Medu']

In [ ]:
# Смотрим получившиеся коррелцицц
df.corr()

# Корреляция между Medu и Fedu стала 63%
# Возможно стоит избавиться от одного из столбцов, либо объедить данные из Medu и Fedu в один столбец.

In [ ]:
df['Medu_Fedu'] = df['Medu'] + df['Fedu']
df.corr()
# Результат объединения
# Корреляция между Medu и score 19%
# Корреляция между Fedu и score 12%
# Коррелиция между объединенным столбцом Medu_Fedu и score 17%
# Для минимизации кол-ва столбцов можно оставить один объединенный столбец.
# Для более точной модели, лучше оставить два отдельных столбца, т. к. разница влияний между ними на итоговый результат велика
df.drop(['Medu_Fedu'], inplace = True, axis = 1)

In [ ]:
# Протестируем все колонки, как категориальные признаки (т. к. их можно назвать таковыми). Тест Стьюдента
for col in df.columns:
    student_test(col)

# Найдены статистически значимые различия для колонок:
# sex, address, Medu, Fedu, Mjob, Fjob, studytime, schoolsup, goout
# Оставим данные колонки в датасете. Medu и Fedu можно будет потом объедить, в случае необходимости сокращения столбцов.
df_for_model = df.loc[:, ['sex', 'address', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'studytime', 'schoolsup', 'goout', 'score']]
df_for_model.head()